# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
gmaps.configure(api_key=g_key)


In [3]:
df = pd.read_csv("../Output/city.csv")
df.head()

,name,temperature,latitude,longitude,humidity,cloudiness,wind_speed
0,meulaboh,84.63,4.1363,96.1285,69,97,0.89
1,vacaville,49.26,38.3566,-121.9877,55,0,16.11
2,beloha,71.40,-25.1667,45.0500,85,88,10.33
3,rio gallegos,42.85,-51.6226,-69.2181,65,20,13.80
4,busselton,71.91,-33.6500,115.3333,39,0,8.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
lat_lng = df[["latitude", "longitude"]]
humidity = df["humidity"]

fig = gmaps.figure(zoom_level = 1, center = (-12.0432,-77.0282))

hm_layer = gmaps.heatmap_layer(
                               lat_lng, weights = humidity, 
                               max_intensity = 300, 
                               dissipating = False, 
                               point_radius = 6
                              )
fig.add_layer(hm_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df.dropna()
new_df = df.loc[(df['temperature'] < 80) & (df['temperature'] > 70) & (df["wind_speed"] < 10)& (df["cloudiness"] == 0)]
new_df

,name,temperature,latitude,longitude,humidity,cloudiness,wind_speed
4,busselton,71.91,-33.6500,115.3333,39,0,8.14
96,san lorenzo,72.59,-25.3333,-57.5333,83,0,5.01
118,dawlatabad,75.42,36.4141,64.9053,28,0,1.32
194,khash,74.14,28.2211,61.2158,10,0,3.53
218,abu dhabi,79.92,24.4667,54.3667,35,0,2.30
384,san angelo,77.13,31.4638,-100.4370,19,0,4.00
418,victor harbor,77.68,-35.5667,138.6167,37,0,9.24
472,iranshahr,79.14,27.2025,60.6848,10,0,0.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
new_df['Hotel Name'] = ""

<ipython-input-6-69085800091b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Hotel Name'] = ""


In [7]:
pars = {
    "key":g_key,
    "radius":5000,
    "type":"lodging"
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for idx, row in new_df.iterrows():
    pars["location"]=f"{row['latitude']},{row['longitude']}"
    
    resp = requests.get(base_url, params=pars).json()
    
    try:
        new_df.loc[idx, "Hotel Name"] = resp["results"][0]["name"]
        print("Hotel found")
    except:
        print("Hotel not found")
    
new_df = new_df.loc[new_df["Hotel Name"] != ""]
new_df

C:\Users\mikej\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found
Hotel found


,name,temperature,latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name
4,busselton,71.91,-33.6500,115.3333,39,0,8.14,Observatory Guest House
96,san lorenzo,72.59,-25.3333,-57.5333,83,0,5.01,Canarias Hotel
118,dawlatabad,75.42,36.4141,64.9053,28,0,1.32,WASIM ASGAR KHAN
194,khash,74.14,28.2211,61.2158,10,0,3.53,خوابگاه آبجی محدثه
218,abu dhabi,79.92,24.4667,54.3667,35,0,2.30,Beach Rotana
384,san angelo,77.13,31.4638,-100.4370,19,0,4.00,"Motel 6 San Angelo, TX"
418,victor harbor,77.68,-35.5667,138.6167,37,0,9.24,McCracken Country Club
472,iranshahr,79.14,27.2025,60.6848,10,0,0.00,نیک سرشت


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_df.iterrows()]
locations = new_df[["latitude", "longitude"]]

In [9]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))